In [1]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

# Open SOS Measurement Dataset

In [2]:
start_date = '20221130'
end_date = '20230509'
# open files
tidy_df_5Min = pd.read_parquet('../sos/tidy_df_20221130_20230517_noplanar_fit.parquet')
tidy_df_30Min = pd.read_parquet('../sos/tidy_df_30Min_20221130_20230517_noplanar_fit.parquet')
# convert time column to datetime
tidy_df_5Min['time'] = pd.to_datetime(tidy_df_5Min['time'])
tidy_df_30Min['time'] = pd.to_datetime(tidy_df_30Min['time'])
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df_5Min = tidy_df_5Min.set_index('time').loc[start_date:end_date].reset_index()
tidy_df_30Min = tidy_df_30Min.set_index('time').loc[start_date:end_date].reset_index()

/var/folders/jy/xv6qnlms2cj_3p5yrzcwc0n00000gn/T/ipykernel_62871/1936205774.py:10: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  tidy_df_5Min = tidy_df_5Min.set_index('time').loc[start_date:end_date].reset_index()


In [22]:
tke_src = tidy_df_5Min[tidy_df_5Min.measurement.isin([
    'turbulent kinetic energy',
    'wind speed'
])].query("tower == 'c'")

In [71]:
tke_wind_src = tke_src.pivot_table(
    index = ['time', 'tower', 'height'],
    columns = 'measurement',
    values = 'value'
)
tke_wind_src.columns = tke_wind_src.columns.to_flat_index()
tke_wind_src = tke_wind_src.reset_index()
tke_wind_src
tke_wind_src['turbulent intensity'] = tke_wind_src['turbulent kinetic energy']**0.5

In [119]:
calm_winter = alt.Chart(
    tke_wind_src.set_index('time').loc['20230131': '20230202'].reset_index(),
    title = 'calm_winter'
).transform_filter(
    alt.FieldOneOfPredicate('height', [3, 10, 20])
).mark_errorband(opacity=0.5).encode(
    alt.X("wind speed:Q").bin(True),
    alt.Y("turbulent intensity:Q"),
    alt.Color("height:N")
).properties(width = 200, height = 200)

blowingsnow_winter = alt.Chart(
    tke_wind_src.set_index('time').loc['20230219': '20230222'].reset_index(),
    title = 'blowingsnow_winter'
).transform_filter(
    alt.FieldOneOfPredicate('height', [3, 10, 20])
).mark_errorband().encode(
    alt.X("wind speed:Q").bin(True),
    alt.Y("turbulent intensity:Q"),
    alt.Color("height:N")
).properties(width = 200, height = 200)

calm_spring = alt.Chart(
    tke_wind_src.set_index('time').loc['20230316': '20230319'].reset_index(),
    title = 'calm_spring'
).transform_filter(
    alt.FieldOneOfPredicate('height', [3, 10, 20])
).mark_errorband().encode(
    alt.X("wind speed:Q").bin(True),
    alt.Y("turbulent intensity:Q"),
    alt.Color("height:N")
).properties(width = 200, height = 200)

(calm_winter | blowingsnow_winter | calm_spring).resolve_scale(y='shared', x='shared')

alt.HConcatChart(...)

In [118]:
calm_winter = alt.Chart(
    tke_wind_src.set_index('time').loc['20230131': '20230202'].reset_index(),
    title = 'calm_winter'
).transform_filter(
    alt.FieldOneOfPredicate('height', [3, 10, 20])
).mark_circle(opacity = 0.5, size=5 ).encode(
    alt.X("wind speed:Q"),
    alt.Y("turbulent intensity:Q"),
    alt.Color("height:N")
).properties(width = 200, height = 200)

blowingsnow_winter = alt.Chart(
    tke_wind_src.set_index('time').loc['20230219': '20230222'].reset_index(),
    title = 'blowingsnow_winter'
).transform_filter(
    alt.FieldOneOfPredicate('height', [3, 10, 20])
).mark_circle(opacity = 0.5, size=5 ).encode(
    alt.X("wind speed:Q"),
    alt.Y("turbulent intensity:Q"),
    alt.Color("height:N")
).properties(width = 200, height = 200)

calm_spring = alt.Chart(
    tke_wind_src.set_index('time').loc['20230316': '20230319'].reset_index(),
    title = 'calm_spring'
).transform_filter(
    alt.FieldOneOfPredicate('height', [3, 10, 20])
).mark_circle(opacity = 0.5, size=5 ).encode(
    alt.X("wind speed:Q"),
    alt.Y("turbulent intensity:Q"),
    alt.Color("height:N")
).properties(width = 200, height = 200)

(calm_winter | blowingsnow_winter | calm_spring).resolve_scale(y='shared', x='shared')

alt.HConcatChart(...)

# Find days with net sublimation in the winter and days without net sublimation in the winter

In [84]:
mean_daily_sublimation = tidy_df_30Min.query("variable == 'w_h2o__3m_c'")[
    ['time', 'value']
].set_index('time').groupby(pd.Grouper(freq='1440Min')).mean().reset_index()
mean_daily_sublimation['net sublimation'] = mean_daily_sublimation['value'] > 0
alt.Chart(mean_daily_sublimation).mark_tick().encode(x='time:T', y = 'net sublimation:N', color='net sublimation:N')

alt.Chart(...)

In [93]:
winter_days = mean_daily_sublimation[mean_daily_sublimation.time.dt.month.isin([12,1])]
winter_days = winter_days[winter_days['net sublimation']]
winter_days['time'] = winter_days['time'].dt.date

spring_days = mean_daily_sublimation[mean_daily_sublimation.time.dt.month.isin([3,4])]
spring_days = spring_days[spring_days['net sublimation']]
spring_days['time'] = spring_days['time'].dt.date

In [107]:
top_winter_days = winter_days.query(f"value > {winter_days['value'].quantile(0.75)}")
top_spring_days = spring_days.query(f"value > {spring_days['value'].quantile(0.75)}")

In [113]:
winter_sublimation_days = alt.Chart(
    tke_wind_src[tke_wind_src.time.dt.date.isin(top_winter_days.time)],
    title = 'winter sublimation days'
).transform_filter(
    alt.FieldOneOfPredicate('height', [3, 10, 20])
).mark_errorband(opacity=0.5).encode(
    alt.X("wind speed:Q").bin(True),
    alt.Y("turbulent intensity:Q"),
    alt.Color("height:N")
).properties(width = 200, height = 200)

spring_sublimation_days = alt.Chart(
    tke_wind_src[tke_wind_src.time.dt.date.isin(top_spring_days.time)],
    title = 'spring sublimation days'
).transform_filter(
    alt.FieldOneOfPredicate('height', [3, 10, 20])
).mark_errorband(opacity=0.5).encode(
    alt.X("wind speed:Q").bin(True),
    alt.Y("turbulent intensity:Q"),
    alt.Color("height:N")
).properties(width = 200, height = 200)


winter_sublimation_days | spring_sublimation_days

alt.HConcatChart(...)

In [147]:
top_winter_sub_days = alt.Chart(
    tke_wind_src[tke_wind_src.time.dt.date.isin(top_winter_days.time)],
    title = 'top winter sublimation days'
).transform_filter(
    alt.FieldOneOfPredicate('height', [3, 10, 20])
# ).mark_circle(opacity = 0.5, size=5 ).encode(
).mark_errorband(opacity=0.5).encode(
    # alt.X("wind speed:Q"),
    alt.X("wind speed:Q").bin(True),
    alt.Y("turbulent intensity:Q"),
    alt.Color("height:N")
).properties(width = 200, height = 200)

top_spring_sub_days = alt.Chart(
    tke_wind_src[tke_wind_src.time.dt.date.isin(top_spring_days.time)],
    title = 'top spring sublimation days'
).transform_filter(
    alt.FieldOneOfPredicate('height', [3, 10, 20])
# ).mark_circle(opacity = 0.5, size=5 ).encode(
).mark_errorband(opacity=0.5).encode(
    # alt.X("wind speed:Q"),
    alt.X("wind speed:Q").bin(True),
    alt.Y("turbulent intensity:Q"),
    alt.Color("height:N")
).properties(width = 200, height = 200)

(top_winter_sub_days | top_spring_sub_days).resolve_scale(y='shared', x='shared')

alt.HConcatChart(...)

In [152]:
top_winter_sub_days = alt.Chart(
    tke_wind_src[tke_wind_src.time.dt.date.isin(top_winter_days.time)],
    title = 'top winter sublimation days'
).transform_filter(
    alt.FieldOneOfPredicate('height', [3, 10, 20])
).mark_circle(opacity = 0.5, size=5 ).encode(
    alt.X("wind speed:Q"),
    alt.Y("turbulent intensity:Q"),
    alt.Color("height:N")
).properties(width = 200, height = 200)

top_spring_sub_days

top_spring_sub_days = alt.Chart(
    tke_wind_src[tke_wind_src.time.dt.date.isin(top_spring_days.time)],
    title = 'top spring sublimation days'
).transform_filter(
    alt.FieldOneOfPredicate('height', [3, 10, 20])
).mark_circle(opacity = 0.5, size=5 ).encode(
    alt.X("wind speed:Q"),
    alt.Y("turbulent intensity:Q"),
    alt.Color("height:N")
).properties(width = 200, height = 200)

(top_winter_sub_days | top_spring_sub_days).resolve_scale(y='shared', x='shared')

alt.HConcatChart(...)

In [144]:
(alt.Chart(
    tke_wind_src[tke_wind_src.time.dt.date.isin(top_winter_days.time)],
    title = 'Winter Days'
).mark_bar().encode(
    alt.X("wind speed:Q").bin(True, maxbins=10),
    alt.Y("count():Q")
).properties(height = 150) & alt.Chart(
    tke_wind_src[tke_wind_src.time.dt.date.isin(top_spring_days.time)],
    title = 'Spring Days'
).mark_bar().encode(
    alt.X("wind speed:Q").bin(True, maxbins=10),
    alt.Y("count():Q")
).properties(height = 150)).resolve_scale(x='shared')

alt.VConcatChart(...)